# 查询构建（Query Construction）

在实际应用中，我们常常需要处理更加复杂和多样化的数据，包括结构化数据（如SQL数据库）、半结构化数据（如带有元数据的文档）以及图数据。

用户的查询也可能不仅仅是简单的语义匹配，而是包含复杂的过滤条件、聚合操作或关系查询。

查询构建利用大语言模型（LLM）的强大理解能力，将用户的自然语言查询“翻译”成针对特定数据源的结构化查询语言或带有过滤条件的请求。

使RAG系统能够无缝地连接和利用各种类型的数据，从而极大地扩展了其应用场景和能力。

![](images/4_2_1.webp)

## 文本到元数据过滤器

构建向量索引时，常常会为文档块（Chunks）附加元数据（Metadata），例如文档来源、发布日期、作者、章节、类别等。

这些元数据为我们提供了在语义搜索之外进行精确过滤的可能。

自查询检索器（Self-Query Retriever） 是LangChain中实现这一功能的核心组件。

工作流程：

1. 定义元数据结构：向LLM清晰地描述文档内容和每个元数据字段的含义及类型。

2. 查询解析：自查询检索器会调用LLM，将用户输入的查询分解为两部分：

    - 查询字符串（Query String）：用于进行语义搜索的部分。

    - 元数据过滤器（Metadata Filter）：从查询中提取出的结构化过滤条件。

3. 执行查询：检索器将解析出的查询字符串和元数据过滤器发送给向量数据库，执行一次同时包含语义搜索和元数据过滤的查询。

例如，对于查询“关于2022年发布的机器学习的论文”，自查询检索器会将其解析为：

- 查询字符串: "机器学习的论文"

- 元数据过滤器: year == 2022

## 示例

以B站视频为例来看看如何使用 SelfQueryRetriever。

In [1]:
# 初始化视频数据
from langchain_community.document_loaders import BiliBiliLoader

import logging
logging.basicConfig(level=logging.INFO)

video_urls = [
    "https://www.bilibili.com/video/BV1Bo4y1A7FU", 
    "https://www.bilibili.com/video/BV1ug4y157xA",
    "https://www.bilibili.com/video/BV1yh411V7ge",
]

bili = []
try:
    # 使用 BiliBiliLoader 加载B站视频的文档和元数据
    loader = BiliBiliLoader(video_urls=video_urls)
    docs = loader.load()
    
    for doc in docs:
        original = doc.metadata
        
        # 手动提取需要的元数据字段
        metadata = {
            'title': original.get('title', '未知标题'),
            'author': original.get('owner', {}).get('name', '未知作者'),
            'source': original.get('bvid', '未知ID'),
            'view_count': original.get('stat', {}).get('view', 0),
            'length': original.get('duration', 0),
        }
        
        # 构建新 metadata 字典
        doc.metadata = metadata
        bili.append(doc)
        
except Exception as e:
    print(f"加载BiliBili视频失败: {str(e)}")

if not bili:
    print("没有成功加载任何视频，程序退出")
    exit()


In [11]:
# 创建向量存储
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="models/bge/bge-small-zh-v1.5")
# 将处理好的文档和元数据存入 Chroma 向量数据库中
vectorstore = Chroma.from_documents(bili, embed_model)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: models/bge/bge-small-zh-v1.5
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [ ]:
# 配置元数据字段信息
# LLM 将依赖这份描述来理解如何处理用户的查询
from langchain_classic.chains.query_constructor.schema import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="视频标题（字符串）",
        type="string", 
    ),
    AttributeInfo(
        name="author",
        description="视频作者（字符串）",
        type="string",
    ),
    AttributeInfo(
        name="view_count",
        description="视频观看次数（整数）",
        type="integer",
    ),
    AttributeInfo(
        name="length",
        description="视频长度，以秒为单位的整数",
        type="integer"
    )
]


In [ ]:
# 配置 LLM
from langchain_deepseek import ChatDeepSeek 

from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0, 
    api_key=os.getenv("Deepseek_API_Key")
    )


In [ ]:
# 配置 SelfQueryRetriever
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="记录视频标题、作者、观看次数等信息的视频元数据",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    verbose=True
)


In [20]:
# 测试 SelfQueryRetriever

queries = [
    "时间最短的视频",
    "时长大于600秒的视频"
]

for query in queries:
    print(f"\n--- 查询: '{query}' ---")
    results = retriever.invoke(query)
    if results:
        for doc in results:
            title = doc.metadata.get('title', '未知标题')
            author = doc.metadata.get('author', '未知作者')
            view_count = doc.metadata.get('view_count', '未知')
            length = doc.metadata.get('length', '未知')
            print(f"标题: {title}")
            print(f"作者: {author}")
            print(f"观看次数: {view_count}")
            print(f"时长: {length}秒")
            print("="*50)
    else:
        print("未找到匹配的视频")



--- 查询: '时间最短的视频' ---


INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain_classic.retrievers.self_query.base:Generated Query: query=' ' filter=None limit=1
INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


标题: 《吴恩达 x OpenAI Prompt课程》【专业翻译，配套代码笔记】02.Prompt 的构建原则
作者: 二次元的Datawhale
观看次数: 19439
时长: 1063秒

--- 查询: '时长大于600秒的视频' ---


INFO:langchain_classic.retrievers.self_query.base:Generated Query: query=' ' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='length', value=600) limit=None


标题: 《吴恩达 x OpenAI Prompt课程》【专业翻译，配套代码笔记】02.Prompt 的构建原则
作者: 二次元的Datawhale
观看次数: 19439
时长: 1063秒
标题: 《吴恩达 x OpenAI Prompt课程》【专业翻译，配套代码笔记】03.Prompt如何迭代优化
作者: 二次元的Datawhale
观看次数: 7355
时长: 806秒


## 文本到Cypher

Cypher 是图数据库（如 Neo4j）中最常用的查询语言，其地位类似于 SQL 之于关系数据库。

与“文本到元数据过滤器”类似，“文本到Cypher”技术利用大语言模型（LLM）将用户的自然语言问题直接翻译成一句精准的 Cypher 查询语句。

LangChain 提供了相应的工具链（如 GraphCypherQAChain），其工作流程通常是：

1. 接收用户的自然语言问题。

2. LLM 根据预先提供的 Schema，将问题转换为 Cypher 查询。

3. 在图数据库上执行该查询，获取精确的结构化数据。

4. 将查询结果再次交由 LLM，生成通顺的自然语言答案。

